# Flipped Classroom 04: Training a Twin Model on MNIST 

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import datetime

%load_ext tensorboard

## Creating a dataset

**Task: take two random mnist digit 28x28x1 images and compute the sum of the numbers that they display.**



In [ ]:
# 1. get mnist from tensorflow_datasets



# 2. preprocess the data using the map method, what else do we need?


# 3. find a way to have two images in each example!


# 4. create the labels for both images


# 4. finish the data pipeline


# 5. check the contents of the data generator

## Create a model for the task

- The two images should be processed using the same parameters (sometimes called a siamese or twin model)

- What do we need?

In [ ]:
class TwinMNISTModel(...):

    # 1. constructor
    def ...
        # inherit functionality from parent class

        # optimizer, loss function and metrics

        # layers to be used


    # 2. call method (forward computation)
    def ...
    



    # 3. metrics property
    def ...
        # return a list with all metrics in the model



    # 4. reset all metrics objects
    def ...



    # 5. train step method
    def ...


        # update the state of the metrics according to loss
        # return a dictionary with metric names as keys and metric results as values





    # 6. test_step method
    def ...
        # same as train step (without parameter updates)






## Create a summary writer to log data

- use tf.summary.create_file_writer(log_path)

In [ ]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

train_log_path = f"logs/"

val_log_path = f"logs/"

train_summary_writer = ...

val_summary_writer = ...

## Write a training loop function

Arguments: 
 - the model to train, 
 - the data to train on, 
 - the data to test on, 
 - how many epochs to train, 
 - the train summary writer object to use for logging
 - the validation summary writer object to use for logging
 - a path to save trained model weights to

In [ ]:
def training_loop(...):

    # 1. iterate over epochs


        # 2. train steps on all batches in the training data



        # 3. log and print training metrics

        with train_summary_writer.as_default():
            ...



        # 4. reset metric objects



        # 5. evaluate on validation data



        # 6. log validation metrics

        with val_summary_writer.as_default():
            ...

        # 7. reset metric objects


    # 8. save model weights


## Use the training loop function to train the model

In [ ]:
# open the tensorboard logs
%tensorboard --logdir logs/

In [ ]:
# 1. instantiate model

# 2. choose a path to save the weights

save_path = ...

# 2. pass arguments to training loop function

training_loop(...)